### Express Prompt: 

The American League Championship Series of Riddler League Baseball determines one of the teams that will compete in the Riddler World Series. This year’s teams — the Tampa Bay Lines and the Minnesota Twin Primes — are evenly matched. In other words, both teams are equally likely to win each game of the best-of-seven series.

On average, how many games will the series last? (Note that the series ends as soon as one team has won four games.)

### Solution: Analytical

- We can solve for expected value based on likelihood of each combinations of games to win
- Each likelihood is `0.5**n`
- Able to solve as "the number of different groups of `r` objects each, which can be formed from the available `n` objects." 
    - Where `n` will represent total games played and `r` will represent 4 each time (e.g. likelihood of `T-F-T-T-T` or `n-4` (for the likelihood of something like `F-F-T-T-F-F`)
- Need to remove any repeated sequences:
        - e.g. `T-T-T-T` is covered in one of the combinations of 5 as `T-T-T-T-F`
        - This can be done by substracting off combination where `n = n-1` and `r=r`

In [1]:
from math import factorial as f
def comb(n, r):
    try:
        return f(n) / (f(n-r) * f(r))
    except:
        return 0 # this is for negative case 

# when we have n = 4 and r = 4 we only have one sequence we can make 
assert(comb(4,4) == 1)

# when we have n = 5 and r = 4 we have 5 total sequences (TTTTF, TFTTT, TTFTT, TTTFT, FTTTT)
assert(comb(5,4) == 5)

# to avoid double counting the same event in n = 5, r = 4 we need to remove TTTTF to get 4 unique
unique_count = comb(5,4) - comb(4,4)
assert(unique_count == 4)

In [2]:
n_vals = range(4,8)
count_dict = {}
for n in n_vals:
    c = 2 * (comb(n,4) - comb(n-1, 4)) # we multiply by 2 to represent each teams combinations at step
    l = c * (0.5**n) # solve likelihood 
    count_dict[n] = {'c': c, 'l': l}

# confirm we have captured all possibilities 
assert(sum([v['l'] for k,v in count_dict.items()]) == 1)

# finally, we simply calculate the expected value 
EV = sum([v['l'] * k for k,v in count_dict.items()])

print(f"The Expected Total Games is {EV:.3f}")


The Expected Total Games is 5.812


### Solution: Simulation

- I just can't help myself - choose a random term until we find 4 `T` or 4 `F` in sequence

In [3]:
import numpy as np
import random
def playSeries(r = 4):
    """We play series until team as Wins = R or Losses = R"""
    arr = np.zeros(shape=(7,1))
    for i in range(7):
        arr[i] = random.choice([0, 1])
        
        # we can check for 4 1s or a sequence of 1s that would indicate 4 0s
        if (sum(arr) == r) or (sum(arr) == i + 1 - r):
            return i + 1

In [4]:
sim_range = [1_000, 10_000, 100_000, 1_000_000]
for sim in sim_range:
    r_sum = 0 # keep running sum 
    for _ in range(sim):
        r_sum += playSeries()
    
    print(f"Played {sim} series: Avg Games to Win was {r_sum / sim:.3f}")

Played 1000 series: Avg Games to Win was 5.799
Played 10000 series: Avg Games to Win was 5.812
Played 100000 series: Avg Games to Win was 5.818
Played 1000000 series: Avg Games to Win was 5.813
